In [ ]:
import torch
from seta import (
    MLPThinker,
    System,
    Dynamics,
    Simulator,
    CustomFunctionDataset,
    Environment,
    Trainer)

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
